# buy-open-sell-close

On the first trading day of each week, buy on the open, then sell on the close.

In [10]:
import pandas as pd
import datetime
import pinkfish as pf

# Format price data.
pd.options.display.float_format = '{:0.2f}'.format

Some global data

In [11]:
symbol = 'SPY'
capital = 10000
start = datetime.datetime(2015, 1, 1)
end = datetime.datetime.now()

Timeseries

In [12]:
# Fetch timeseries, select.
ts = pf.fetch_timeseries(symbol)
ts = pf.select_tradeperiod(ts, start, end, use_adj=False)

# Add calendar columns.
ts = pf.calendar(ts)

# Finalize timeseries.
ts, start = pf.finalize_timeseries(ts, start)

# Create tradelog and daily balance objects.
tlog = pf.TradeLog(symbol)
dbal = pf.DailyBal()

In [13]:
ts

,open,high,low,close,adj_close,volume,dotw,dotm,doty,month,first_dotw,first_dotm,first_doty,last_dotw,last_dotm,last_doty
date,,,,,,,,,,,,,,,,
2015-01-02,206.38,206.88,204.18,205.43,177.46,121465900,4,2,2,1,False,True,True,True,False,False
2015-01-05,204.17,204.37,201.35,201.72,174.26,169632600,0,5,5,1,True,False,False,False,False,False
2015-01-06,202.09,202.72,198.86,199.82,172.62,209151400,1,6,6,1,False,False,False,False,False,False
2015-01-07,201.42,202.72,200.88,202.31,174.77,125346700,2,7,7,1,False,False,False,False,False,False
2015-01-08,204.01,206.16,203.99,205.90,177.87,147217800,3,8,8,1,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-12-22,383.05,386.21,374.77,380.72,380.72,100120900,3,22,356,12,False,False,False,False,False,False
2022-12-23,379.65,383.06,378.03,382.91,382.91,59761100,4,23,357,12,False,False,False,True,False,False
2022-12-27,382.79,383.15,379.65,381.40,381.40,51638200,1,27,361,12,True,False,False,False,False,False


Algorithm

In [14]:
pf.TradeLog.cash = capital

# Loop through timeseries.
for i, row in enumerate(ts.itertuples()):

    date = row.Index.to_pydatetime()
    end_flag = pf.is_last_row(ts, i)

    # On first trading day of the week, buy open, then sell close.
    if row.first_dotw:
        tlog.buy(date, row.open)
        tlog.sell(date, row.close)

    # Record daily balance.
    dbal.append(date, row.close, row.high, row.low)

Retrieve logs

In [15]:
tlog = tlog.get_log()
dbal = dbal.get_log(tlog)

In [16]:
tlog.tail()

,entry_date,entry_price,exit_date,exit_price,pl_points,pl_cash,qty,cumul_total,direction,symbol
412,2022-11-28,399.09,2022-11-28,395.91,-3.18,-85.86,27,814.55,LONG,SPY
413,2022-12-05,403.95,2022-12-05,399.59,-4.36,-113.36,26,701.19,LONG,SPY
414,2022-12-12,394.11,2022-12-12,398.95,4.84,130.68,27,831.87,LONG,SPY
415,2022-12-19,383.47,2022-12-19,380.02,-3.45,-96.60,28,735.27,LONG,SPY
416,2022-12-27,382.79,2022-12-27,381.40,-1.39,-38.92,28,696.35,LONG,SPY


Get stats

In [17]:
stats = pf.stats(ts, tlog, dbal, capital)

Summary

In [18]:
pf.summary(stats)

,strategy
annual_return_rate,0.85
max_closed_out_drawdown,-13.98
best_month,6.04
worst_month,-5.08
sharpe_ratio,0.18
sortino_ratio,0.11
monthly_std,1.43
annual_std,6.15
